# 科学技術計算 5 レポート


In [1]:
import numpy as np
import scipy

from numpy.linalg import inv, norm, solve, det, matrix_rank, cond, pinv, lstsq, eig, eigh, eigvals, eigvalsh
from numpy import diag
rng = np.random.default_rng()

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"

from typing import List, Optional, Union, Tuple

import sklearn
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces

from matplotlib.pyplot import imshow


## 課題05-1

In [3]:
#レイリー商反復について
m = 10
n = 5

# 対称行列Aを生成
A = rng.random(size=(m, n))
A = A.T @ A  # Aを対称行列にする
print("symmetric matrix A\n", A, sep='')

u = np.ones(n)      #固有値ベクトル生成(すべて1のベクトル)
mu = 0.5            #シフト値
print("mu", mu)

#関数
def rayleigh_quotient_iteration(A: np.ndarray,mu: float, u: np.ndarray = np.ones(n), maxiter: int = 5000,) -> Tuple[float, np.ndarray]:
    """Rayleigh quotioent iteration for computing the largest eigenvalue and its eigenvector of A

    Args:
        A (np.ndarray): nxn matrix A
        mu (float): initial eigenvalue
        u (np.ndarray): n-d initial eigenvector
        maxiter (int, optional): max iterations. Defaults to 50.

    Returns:
        Tuple[float, np.ndarray]: the largest eigenvalue and its eigenvector
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    assert len(u) == n

    u_pre = u.copy()
    for i in range(maxiter):
        u = solve(A - mu * np.eye(n), u)
        u /= norm(u)
        mu = u.T @ A @ u

        with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
            print(f"{i:2d}", u)

        if np.allclose(u, u_pre) or np.allclose(u, -u_pre):
            break
        u_pre = u.copy()

    return mu, u


symmetric matrix A
[[4.47767472 2.70863858 3.57333675 3.64327128 2.59535967]
 [2.70863858 2.28659739 2.48690396 2.33552591 1.77230104]
 [3.57333675 2.48690396 4.17383484 3.26348219 2.98222212]
 [3.64327128 2.33552591 3.26348219 3.51308023 2.55855994]
 [2.59535967 1.77230104 2.98222212 2.55855994 2.94988519]]
mu 0.5
 0 0.459627346367 [ 0.116729599733 -0.337327097426  0.771564465397 -0.213479586643 -0.481351609233]
 1 0.414355025416 [-0.262408711886  0.755542761290 -0.482862007040 -0.048588650358  0.353242450736]
 2 0.407734121405 [ 0.207911680188 -0.635207822064  0.602192845234 -0.041086924327 -0.434694602944]
 3 0.407720833806 [-0.210008245899  0.640955827907 -0.597750334239  0.036781666287  0.431756656311]
 4 0.407720833805 [ 0.210008034409 -0.640955342978  0.597750715658 -0.036781981457 -0.431756924163]
 5 0.407720833805 [-0.210008034409  0.640955342978 -0.597750715658  0.036781981457  0.431756924163]
 6 0.407720833805 [ 0.210008034409 -0.640955342978  0.597750715658 -0.036781981457 

In [ ]:
#QR法関数
A0 = A.copy()

Qall = np.eye(A0.shape[0])  # 直交行列の積Q_xを格納するための初期値
maxiter = 10

for i in range(maxiter):
    # QR法の反復
    Q, R = np.linalg.qr(A0)
    A0 = R @ Q

    # 直交行列Qの積を格納
    Qall = Qall @ Q

    print()
    print(i, diag(A0))
    QtAQ = Qall.T @ A @ Qall

    print("Q^T A Q == A^(k+1)", np.allclose(QtAQ, A0))

    print(
        "max of abs of super diagonals of Q^T A Q",
        np.abs(QtAQ[np.tril_indices(n, k=1)]).max()
    )

    with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        print("Q^T A Q\n", QtAQ, sep='')


def eigh_by_qr_method(
    A0: np.ndarray,
    maxiter: int = 1000,
    qr_decomp: callable = np.linalg.qr
) -> Tuple[np.ndarray, np.ndarray]:
    """Computing eigenvalues and eigenvectors by QR decompoition

    Args:
        A0 (np.ndarray): nxn matrix
        maxiter (int, optional): max iterations. Defaults to 1000.
        qr_decomp (callable, optional): QR decomposition solver. Defaults to np.linalg.qr.

    Returns:
        Tuple[np.ndarray, np.ndarray]: n-d vector of eigenvalues and orthogonal matrix with eigenvector columns
    """
    assert A0.ndim == 2
    assert A0.shape[0] == A0.shape[1]
    n = A0.shape[0]
    A = A0.copy()

    Qall = np.eye(n)
    diagonals_pre = diag(A)
    superdiagonals_pre = diag(A, k=1)

    for i in range(maxiter):
        Q, R = qr_decomp(A)
        A = R @ Q
        Qall = Qall @ Q

        diagonals = diag(A)
        superdiagonals = diag(A, k=1)
        with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
            print(f"     diagonals {i:2d}", diagonals)
            print(f"superdiagonals {i:2d}", superdiagonals)

        if (np.allclose(diagonals, diagonals_pre) and
            np.allclose(superdiagonals, superdiagonals_pre)):
            with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
                print("A\n", A, sep='')
            break

        diagonals_pre = diagonals.copy()
        superdiagonals_pre = superdiagonals.copy()

    return diagonals, Qall

with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
    print("using numpy QR")
    d, U = eigh_by_qr_method(A)
    print("d", d)
    print("U\n", U)
    # print("using scipy QR")
    # d, U = eigh_by_qr_method(A, qr_decomp=scipy.linalg.qr)
    # print(d)

with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    d_np, U_np = np.linalg.eigh(A)
    U_np = U_np[:, ::-1]
    d_np = d_np[::-1]

    print("d_np", d_np)
    print("d   ", d)
    print("d == d_np", np.allclose(d, d_np))

    print("U_np\n", U_np, sep='')
    print("U\n", U, sep='')

    for i in range(len(U)):
        print(
            f"column {i}: U[{i}] == U_np[{i}] ?",
            np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
        )



In [ ]:
#ハウスホルダー法
def householder_step(
    A: np.ndarray,
    k: int
) -> Tuple[np.ndarray, np.ndarray]:
    """k-th step of Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix
        k (int): step

    Returns:
        Tuple[np.ndarray, np.ndarray]: transformed matrix QAQ after k-th step, and the transfrom Q
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]

    n = A.shape[0]
    assert 0 <= k < n - 2

    x = A[k + 1:, k]  # Aのk列目のk+1行目以降の部分を取得
    y = np.zeros_like(x)
    # y[0] = - np.sign(A[k + 1, k]) * norm(x)
    y[0] = - np.sign(x[0]) * norm(x)  # ベクトルyの先頭要素を設定

    print("x", x)
    print("y", y)

    u = (x - y) / norm(x - y)  # 単位ベクトルuを計算
    H = np.eye(len(x)) - 2 * np.outer(u, u)  # ハウスホルダー行列Hを計算

    Q = np.eye(n)
    Q[k + 1:, k + 1:] = H  # QにHを組み込む

    QAQ = Q @ A @ Q  # 変換後の行列を計算
    return QAQ, Q

def householder_transform(
    A: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray]:
    """Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: tridiagonal Atri and orthogonal Qhh
                                       so that Qhh A Qhh == Atri
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    Atri = A.copy()
    Qhh = np.eye(n)
    for k in range(n - 2):
        print(f"{k}-th step")
        Atri, Q = householder_step(Atri, k)
        Qhh = Qhh @ Q  # 直交行列を更新
        print("Atri\n", Atri, sep='')
        print("Q\n", Q, sep='')
        print("Qhh\n", Qhh, sep='')
    return Atri, Qhh


with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):

    print("A\n", A, sep='')
    Atri, Qhh = householder_transform(A)

    print("tridiagonalized Atri\n", Atri, sep='')
    print("check Q.T A Q\n", Qhh.T @ A @ Qhh)
    print("Q.T A Q == Atri", np.allclose(Qhh.T @ A @ Qhh, Atri))

with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    Q, R = np.linalg.qr(Atri)
    print("Atri\n", Atri, sep='')
    print("Q\n", Q, sep='')
    print("R\n", R, sep='')
    print("RQ\n", R @ Q, sep='')



In [ ]:
def qr_decom_by_givens(
    Atri_org: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray]:
    """QR decomposition of a tridiagonal matrix by Givens rotation

    Args:
        Atri_org (np.ndarray): an nxn tridiagonal matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: orthogonal Q and upper traiangular R so that A == QR
    """
    assert len(Atri_org.shape) == 2
    assert Atri_org.shape[0] == Atri_org.shape[1]
    # assert AAtri_org should be tridiagonal
    n = Atri_org.shape[0]

    Atri = Atri_org.copy()
    Q = np.eye(n)

    for k in range(n - 1):
        p, q = k, k + 1
        Rot2x2 = givens_r_matrix(Atri[[p, q]][:, p])
        Atri[[p, q]] = Rot2x2.T @ Atri[[p, q]]
        Q[:, [p, q]] = Q[:, [p, q]] @ Rot2x2

    R = Atri
    return Q, R


## 課題05-3